In [56]:
from dotenv import load_dotenv
from pathlib import Path
import os
import sys
import openai
from typing import Optional, Union


env_path = Path('../..') / '.env'
load_dotenv(dotenv_path=env_path)

print(os.getcwd())
sys.path.append('/Users/davidwei/Documents/SellScale/sellscale-api/notebooks/research')

OPENAI_API_KEY = os.environ.get('OPENAI_API_KEY')
openai.api_key = OPENAI_API_KEY

/Users/davidwei/Documents/SellScale/sellscale-api/notebooks/research


In [84]:
# Taken from src/ml/openai_wrappers.py
CURRENT_OPENAI_DAVINCI_MODEL = "text-davinci-003"
CURRENT_OPENAI_CHAT_GPT_MODEL = "gpt-3.5-turbo"
CURRENT_OPENAI_LATEST_GPT_MODEL = "gpt-4"
DEFAULT_SUFFIX = None
DEFAULT_MAX_TOKENS = 16
DEFAULT_TEMPERATURE = 1
DEFAULT_TOP_P = 1
DEFAULT_N = 1
DEFAULT_FREQUENCY_PENALTY = 0
DEFAULT_STOP = None

def wrapped_chat_gpt_completion(
    messages: list,
    history: Optional[list] = [],
    max_tokens: Optional[int] = DEFAULT_MAX_TOKENS,
    temperature: Optional[float] = DEFAULT_TEMPERATURE,
    top_p: Optional[float] = DEFAULT_TOP_P,
    n: Optional[int] = DEFAULT_N,
    frequency_penalty: Optional[float] = DEFAULT_FREQUENCY_PENALTY,
    stop: Optional[Union[str, list]] = DEFAULT_STOP,
) -> str:
    """
    Generates a completion using the GPT-3.5-turbo model.

    messages needs to be in the format:
    [
        {
            "role": "user",
            "content": "Hello, how are you?"
        },
        {
            "role": "assistant",
            "content": "I am doing well, how about you?"
        }
        ...
    ]
    """
    if history:
        messages = history + messages

    response = openai.ChatCompletion.create(
        model=CURRENT_OPENAI_CHAT_GPT_MODEL,
        messages=messages,
        max_tokens=max_tokens,
        temperature=temperature,
        top_p=top_p,
        n=n,
        frequency_penalty=frequency_penalty,
        stop=stop,
    )
    if response is None or response["choices"] is None or len(response["choices"]) == 0:
        return ""

    choices = response["choices"]
    top_choice = choices[0]
    preview = top_choice["message"]["content"].strip()

    messages = messages + [{"role": "assistant", "content": preview}]
    return messages, preview


In [100]:
def get_research_paragraph_form() -> str:
    prompt: str = """I am a sales researcher who is identifying why the prospect company would be interested in purchasing my company, {sdr_company_name}'s, product or service.

This is what my company, {sdr_company_name}, does:
- tagline: {sdr_company_tagline}
- description: {sdr_company_description}

I am selling to a prospect named '{prospect_name}' who works at a company called '{prospect_company_name}'. Here is information about the prospect and the account I am selling to:
- Prospect Name: {prospect_name}
- Prospect Title: {prospect_title}
- Prospect Persona: {prospect_persona_name}
- Prospect Bio: {prospect_bio}
- Persona Description: {prospect_persona_description}
- Persona Buy Reason: {prospect_persona_buy_reason}
- Company Name: {prospect_company_name}
- Company Tagline: {prospect_company_tagline}
- Company Description: {prospect_company_description}
- Company Size: {prospect_company_size}

Based on this information, give me a detailed report as to why {prospect_name} and {prospect_company_name} would want to buy {sdr_company_name}'s product.

Ensure you relate each point to {prospect_name} and {prospect_company_name} and be very specific.""".format(
    sdr_company_name="AskEdith",
    sdr_company_tagline="The Natural Language Interface for Databases",
    sdr_company_description="Unlock the full potential of your data with search-based analytics powered by AI. Easily navigate and analyze large sets of information to make informed business decisions. Say goodbye to data silos and hello to actionable insights with search-based analytics.",
    prospect_name="Mani Ramaswamy",
    prospect_title="Chief Technology Officer",
    prospect_persona_name="CTOs at Tech Companies",
    prospect_bio="Mani has over 25 years of experience in the industry!",
    prospect_persona_description="The CTO at Tech Companies persona is a tech-savvy professional who is responsible for directing the technology strategy and operations of their organization. They have a keen interest in leveraging innovative tools and technologies, such as ChatGPT and GPT-4, to streamline business operations and drive growth. With a deep understanding of data management, AI, and search-based analytics, they are constantly seeking new ways to unlock the full potential of their organization's data. This persona is focused on making informed business decisions based on insights gleaned from large sets of information, and they turn to AskEdith for its intuitive and powerful natural language interface for databases. With AskEdith, the CTO at Tech Companies can easily navigate and analyze data sets, break down data silos, and achieve actionable insights that drive the success of their organization.",
    prospect_persona_buy_reason="As a CTO at a tech company, managing large sets of data and gaining actionable insights is a critical part of their job. With AskEdith's natural language interface for databases, this persona can easily navigate and analyze their company's data to make informed business decisions. This means that they can unlock the full potential of their data and eliminate data silos, ultimately saving time and resources. Additionally, being powered by AI means that AskEdith can surface insights and trends that may have otherwise gone unnoticed, giving this persona a competitive edge in their industry.",
    prospect_company_name="Auritas",
    prospect_company_tagline="Leaders in Data & Process Management.",
    prospect_company_description="Auritas is a global SAP consulting and implementation services company specializing in sophisticated approaches to Information Lifecycle Management (ILM), Enterprise Content Management (ECM), and SAP HANA Optimization.",
    prospect_company_size="500-1000 employees",
)

    history, completion = wrapped_chat_gpt_completion(
        messages=[
            {
                "role": "user",
                "content": prompt
            },
        ],
        max_tokens=512,
    )

    return history, completion

In [101]:
history, completion = get_research_paragraph_form()
print(history)
print(completion)

[{'role': 'user', 'content': "I am a sales researcher who is identifying why the prospect company would be interested in purchasing my company, AskEdith's, product or service.\n\nThis is what my company, AskEdith, does:\n- tagline: The Natural Language Interface for Databases\n- description: Unlock the full potential of your data with search-based analytics powered by AI. Easily navigate and analyze large sets of information to make informed business decisions. Say goodbye to data silos and hello to actionable insights with search-based analytics.\n\nI am selling to a prospect named 'Mani Ramaswamy' who works at a company called 'Auritas'. Here is information about the prospect and the account I am selling to:\n- Prospect Name: Mani Ramaswamy\n- Prospect Title: Chief Technology Officer\n- Prospect Persona: CTOs at Tech Companies\n- Prospect Bio: Mani has over 25 years of experience in the industry!\n- Persona Description: The CTO at Tech Companies persona is a tech-savvy professional w

In [107]:
def get_research_bullet_form(history: list) -> str:
    prompt = """Convert this report into a set of 4-6 highly specific research points
- Each research point needs a 'source' and 'reason'
- Each research point must explicitly mention {prospect_name} and/or {prospect_company_name}, with high priority given to {prospect_name}.
- The source is a one to two word phrase that describes where the information may have come from.
- The reason is a short sentence that synthesizes why {prospect_name} and {prospect_company_name} should buy {sdr_company_name}'s product.

Follow the format:
- Source: source
- Reason: reason


Keep the bullet points short and concise while ensuring that they are highly specific to {prospect_name} and {prospect_company_name}.
    """.format(
        sdr_company_name="AskEdith",
        prospect_name="Mani Ramaswamy",
        prospect_company_name="Auritas"
    )

    history, completion = wrapped_chat_gpt_completion(
        messages=[
            {
                "role": "user",
                "content": prompt
            },
        ],
        history=history,
        max_tokens=512,
    )

    return history, completion



In [102]:
history, paragraph = get_research_paragraph_form()
history, bullet_points = get_research_bullet_form(history)
print(history)
print(bullet_points)

[{'role': 'user', 'content': "I am a sales researcher who is identifying why the prospect company would be interested in purchasing my company, AskEdith's, product or service.\n\nThis is what my company, AskEdith, does:\n- tagline: The Natural Language Interface for Databases\n- description: Unlock the full potential of your data with search-based analytics powered by AI. Easily navigate and analyze large sets of information to make informed business decisions. Say goodbye to data silos and hello to actionable insights with search-based analytics.\n\nI am selling to a prospect named 'Mani Ramaswamy' who works at a company called 'Auritas'. Here is information about the prospect and the account I am selling to:\n- Prospect Name: Mani Ramaswamy\n- Prospect Title: Chief Technology Officer\n- Prospect Persona: CTOs at Tech Companies\n- Prospect Bio: Mani has over 25 years of experience in the industry!\n- Persona Description: The CTO at Tech Companies persona is a tech-savvy professional w

In [109]:
def gpt_convert_to_json(history: list):
    prompt: str = """Convert these research points into a JSON object. 

    The JSON object should be a list of dictionaries, with each dictionary containing a 'source' and 'reason' key.
    
    The format should be as following:

    ```
    [
        {
            "source": "source",
            "reason": "reason"
        },
    ]
    ```
    """

    history, completion = wrapped_chat_gpt_completion(
        messages=[
            {
                "role": "user",
                "content": prompt
            },
        ],
        history=history,
        max_tokens=512,
    )

    return history, completion

    

In [110]:
history, paragraph = get_research_paragraph_form()
history, bullet_points = get_research_bullet_form(history)
history, json = gpt_convert_to_json(history)
print(history)
print(json)



[{'role': 'user', 'content': "I am a sales researcher who is identifying why the prospect company would be interested in purchasing my company, AskEdith's, product or service.\n\nThis is what my company, AskEdith, does:\n- tagline: The Natural Language Interface for Databases\n- description: Unlock the full potential of your data with search-based analytics powered by AI. Easily navigate and analyze large sets of information to make informed business decisions. Say goodbye to data silos and hello to actionable insights with search-based analytics.\n\nI am selling to a prospect named 'Mani Ramaswamy' who works at a company called 'Auritas'. Here is information about the prospect and the account I am selling to:\n- Prospect Name: Mani Ramaswamy\n- Prospect Title: Chief Technology Officer\n- Prospect Persona: CTOs at Tech Companies\n- Prospect Bio: Mani has over 25 years of experience in the industry!\n- Persona Description: The CTO at Tech Companies persona is a tech-savvy professional w